In [27]:
import pandas as pd
import numpy as np



#### my guess of what the issue is currently the food can't be matched with the libre GL?
perhaps i can approximate with dexcom readings of the food and \
guess on the relative scale of the participants in that dataset and approximate that the same food in this dataset(cgmarcos) \
has the same relative change in glucose given features of the participant (particant_weight_cgm == particant_weight_big_ideas, etc.))

In [21]:
# path of  where the original data is to find a distinct dummy variable
# for pre dia , dia, and non
# C:\Users\najer\OneDrive - UC San Diego\winter_2025\final_project_data\data\cgmacros-a-scientific-dataset-for-personalized-nutrition-and-diet-monitoring-1.0.0

In [4]:
dexcom_equivalent = pd.read_csv("cleaned_data/CGMacros-001.csv")

In [61]:
%ls

 Volume in drive C is Local Disk
 Volume Serial Number is 7C98-7A30

 Directory of C:\Users\najer\OneDrive - UC San Diego\winter_2025\dsc_106\project_4\Final-Project

03/02/2025  12:05 PM    <DIR>          .
02/24/2025  07:40 PM    <DIR>          ..
03/01/2025  09:25 AM                41 .gitignore
03/01/2025  11:29 PM    <DIR>          .ipynb_checkpoints
03/01/2025  11:29 PM    <DIR>          cleaned_data
03/01/2025  11:08 PM    <DIR>          data
03/01/2025  09:25 AM             2,709 data_cleaning.py
02/28/2025  11:58 AM    <DIR>          eda
03/02/2025  12:05 PM            75,902 felix_refactor_dexcom.ipynb
03/01/2025  09:25 AM               458 global.js
03/01/2025  09:25 AM               270 index.html
03/01/2025  09:25 AM               190 README.md
03/02/2025  12:00 PM               236 TODO.md
               7 File(s)         79,806 bytes
               6 Dir(s)  351,498,616,832 bytes free


In [59]:
dexcom_equivalent.head()

,Timestamp,Libre GL,Dexcom GL,HR,Calories (Activity),METs,Meal Type,Calories,Carbs,Protein,Fat,Fiber,Amount Consumed
0,2020-05-01 10:30:00,84.000000,NaN,56.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-01 10:31:00,84.133333,NaN,56.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-01 10:32:00,84.266667,NaN,57.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-01 10:33:00,84.400000,NaN,54.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-01 10:34:00,84.533333,NaN,55.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# (dexcom_equivalent["Timestamp"][0]).month
# i think the script is currently missing dropping the 11th day and making timestampe to_datetime?

AttributeError: 'str' object has no attribute 'month'

In [18]:
(dexcom_equivalent["Libre GL"]).value_counts(dropna=False).sum()

np.int64(14730)

In [19]:
(dexcom_equivalent["Dexcom GL"]).value_counts(dropna=False).sum()

np.int64(14730)

#### i think it be fine if we relinquish few hundred points per participant unless we do a prediction of the free space available in the groups
_____

In [22]:
%pwd

'C:\\Users\\najer\\OneDrive - UC San Diego\\winter_2025\\dsc_106\\project_4\\Final-Project'

In [23]:
%ls

 Volume in drive C is Local Disk
 Volume Serial Number is 7C98-7A30

 Directory of C:\Users\najer\OneDrive - UC San Diego\winter_2025\dsc_106\project_4\Final-Project

03/02/2025  10:48 AM    <DIR>          .
02/24/2025  07:40 PM    <DIR>          ..
03/01/2025  09:25 AM                41 .gitignore
03/01/2025  11:29 PM    <DIR>          .ipynb_checkpoints
03/01/2025  11:29 PM    <DIR>          cleaned_data
03/01/2025  11:08 PM    <DIR>          data
03/01/2025  09:25 AM             2,709 data_cleaning.py
02/28/2025  11:58 AM    <DIR>          eda
03/02/2025  10:48 AM             3,220 felix_refactor_dexcom.ipynb
03/01/2025  09:25 AM               458 global.js
03/01/2025  09:25 AM               270 index.html
03/01/2025  09:25 AM               190 README.md
               6 File(s)          6,888 bytes
               6 Dir(s)  351,614,066,688 bytes free


#### we can make groups of  A1c (this is how pre dia, dia, non is declrared)




| normal      | Prediabetes     | diabetes |
| ------------- | ------------- | ------- |
| below 5.7% | : 5.7% to 6.4% | Diabetes: 6.5% or above | 
| *units in A1c*

In [48]:
filtering_participants = pd.read_csv("cleaned_data/bio.csv")

In [53]:
filtering_participants.head()

,PID,Age,Gender,BMI,Body weight,Height,Self-identifid-Race,A1c PDL (Lab),Fasting GLU - PDL (Lab),Insulin,...,HDL,Non HDL,LDL (Cal),VLDL (Cal),Cho/HDL Ratio,Collection time PDL (Lab),Starting GLU,Normal,Prediabetes,Diabetes
0,1,27,M,22.265239,133.8,65.0,Hispanic/Latino,5.4,91,2.5,...,74,142,130,13,2.9,11:06:00 AM,81.000000,1,0,0
1,2,49,F,30.946742,169.2,62.0,Hispanic/Latino,5.5,93,14.8,...,91,90,78,12,2.0,7:38:00 AM,98.000000,1,0,0
2,3,59,F,26.948690,157.0,64.0,Hispanic/Latino,6.5,118,17.4,...,74,116,90,31,2.6,7:25:00 AM,127.666667,0,0,1
3,4,33,F,42.384279,262.6,66.0,Hispanic/Latino,5.5,105,19.4,...,46,221,164,60,5.8,7:20:00 AM,103.000000,1,0,0
4,5,51,F,30.957534,172.0,62.5,Hispanic/Latino,6.6,144,12.9,...,38,231,157,78,7.1,7:45:00 AM,161.333333,0,0,1


In [44]:
filtering_participants["A1c PDL (Lab)"]

0     5.4
1     5.5
2     6.5
3     5.5
4     6.6
5     5.2
6     5.9
7     5.8
8     5.7
9     5.7
10    5.7
11    7.1
12    6.2
13    7.1
14    5.4
15    5.8
16    5.0
17    5.4
18    5.6
19    6.4
20    5.3
21    5.9
22    6.2
23    5.9
24    4.7
25    7.0
26    6.1
27    7.6
28    5.1
29    5.3
30    5.6
31    4.6
32    8.5
33    6.9
34    7.2
35    8.3
36    6.3
37    7.1
38    6.0
39    6.0
40    6.1
41    7.4
42    6.9
43    4.8
44    7.2
Name: A1c PDL (Lab), dtype: float64

In [50]:


def bin_a1c(df):
    df['Normal'] = np.where(df['A1c PDL (Lab)'] < 5.7, 1, 0)
    df['Prediabetes'] = np.where((df['A1c PDL (Lab)'] >= 5.7) & (df['A1c PDL (Lab)'] < 6.5), 1, 0)
    df['Diabetes'] = np.where(df['A1c PDL (Lab)'] >= 6.5, 1, 0)
    return df






In [51]:
participants_group = bin_a1c(filtering_participants)
participants_group


,PID,Age,Gender,BMI,Body weight,Height,Self-identifid-Race,A1c PDL (Lab),Fasting GLU - PDL (Lab),Insulin,...,HDL,Non HDL,LDL (Cal),VLDL (Cal),Cho/HDL Ratio,Collection time PDL (Lab),Starting GLU,Normal,Prediabetes,Diabetes
0,1,27,M,22.265239,133.8,65.00,Hispanic/Latino,5.4,91,2.5,...,74,142,130,13,2.9,11:06:00 AM,81.000000,1,0,0
1,2,49,F,30.946742,169.2,62.00,Hispanic/Latino,5.5,93,14.8,...,91,90,78,12,2.0,7:38:00 AM,98.000000,1,0,0
2,3,59,F,26.948690,157.0,64.00,Hispanic/Latino,6.5,118,17.4,...,74,116,90,31,2.6,7:25:00 AM,127.666667,0,0,1
3,4,33,F,42.384279,262.6,66.00,Hispanic/Latino,5.5,105,19.4,...,46,221,164,60,5.8,7:20:00 AM,103.000000,1,0,0
4,5,51,F,30.957534,172.0,62.50,Hispanic/Latino,6.6,144,12.9,...,38,231,157,78,7.1,7:45:00 AM,161.333333,0,0,1
5,6,51,F,29.303451,197.0,68.75,White,5.2,96,6.4,...,72,131,118,15,2.8,7:45:00 AM,88.333333,1,0,0
6,7,66,F,27.070327,199.6,72.00,Hispanic/Latino,5.9,108,15.9,...,43,85,67,18,3.0,8:00:00 AM,122.333333,0,1,0
7,8,54,M,39.945440,218.4,62.00,Hispanic/Latino,5.8,112,17.7,...,60,120,95,29,3.0,8:55:00 AM,120.000000,0,1,0
8,9,34,F,37.001506,183.2,59.00,Hispanic/Latino,5.7,122,25.7,...,40,162,108,62,5.1,7:32:00 AM,114.333333,0,1,0
9,10,54,F,35.811892,195.8,62.00,Hispanic/Latino,5.7,100,15.3,...,55,128,110,20,3.3,7:44:00 AM,122.666667,0,1,0


#### im using dummy variables since we'll eventually need to use decision trees when the quiz is implemented

In [54]:
%pwd

'C:\\Users\\najer\\OneDrive - UC San Diego\\winter_2025\\dsc_106\\project_4\\Final-Project'